In [ ]:
import numpy as np
import suite2p
import matplotlib.pyplot as plt
from pathlib import Path
from typing import Sequence, Dict, Any
from scipy.stats import kde, pearsonr
import numpy as np
import pandas as pd
import os
import pickle
from scipy.signal import find_peaks
import cottage_analysis as cott
from cottage_analysis.imaging.common import align_timestamps, find_frames

# Suite2p visualization

## Filepath

In [ ]:
# filepath
rawdata_root = "/camp/lab/znamenskiyp/data/instruments/raw_data/projects/"
root = "/camp/lab/znamenskiyp/home/shared/projects/"
project = "hey2_3d-vision_20210716"
data_dir = ""
mouse = "PZAH2.1b"
session = "S20210916"
recording = "R163430"
protocol = "SphereSparseNoise"
suite2p_dir = "suite2p_rois_0/suite2p/plane0/"
trace_dir = "suite2p_traces_0/"


def generate_filefolder(
    root, rawdata_root, project, data_dir, mouse, session, recording, protocol
):
    rawdata_folder = (
        rawdata_root
        + project
        + "/"
        + data_dir
        + mouse
        + "/"
        + session
        + "/"
        + recording
        + "_"
        + protocol
        + "/"
    )
    preprocess_folder = (
        root
        + project
        + "/"
        + data_dir
        + mouse
        + "/"
        + session
        + "/"
        + recording
        + "_"
        + protocol
        + "/"
    )
    analysis_folder = (
        root
        + project
        + "/"
        + data_dir
        + "Analysis/"
        + mouse
        + "/"
        + session
        + "/"
        + recording
        + "_"
        + protocol
        + "/"
    )

    return rawdata_folder, preprocess_folder, analysis_folder


def generate_suite2p_folder(
    root, project, data_dir, mouse, session, recording, protocol, suite2p_dir, trace_dir
):
    suite2p_folder = (
        root + project + "/" + data_dir + mouse + "/" + session + "/" + suite2p_dir
    )
    protocol_folder = (
        root
        + project
        + "/"
        + data_dir
        + mouse
        + "/"
        + session
        + "/"
        + recording
        + "_"
        + protocol
        + "/"
    )
    trace_folder = protocol_folder + protocol + "_" + trace_dir

    return suite2p_folder, protocol_folder, trace_folder


rawdata_folder, preprocess_folder, analysis_folder = generate_filefolder(
    root, rawdata_root, project, data_dir, mouse, session, recording, protocol
)
print(rawdata_folder, preprocess_folder, analysis_folder)

suite2p_folder, protocol_folder, trace_folder = generate_suite2p_folder(
    root, project, data_dir, mouse, session, recording, protocol, suite2p_dir, trace_dir
)
print(suite2p_folder, protocol_folder, trace_folder)

if not os.path.exists(analysis_folder):
    os.makedirs(analysis_folder)
print(trace_folder)

## Load files

In [ ]:
# Load files
F = np.load(suite2p_folder + "F.npy", allow_pickle=True)
Fneu = np.load(suite2p_folder + "Fneu.npy", allow_pickle=True)
spks = np.load(suite2p_folder + "spks.npy", allow_pickle=True)
stat = np.load(suite2p_folder + "stat.npy", allow_pickle=True)
ops = np.load(suite2p_folder + "ops.npy", allow_pickle=True)
ops = ops.item()
iscell = np.load(suite2p_folder + "iscell.npy", allow_pickle=True)[:, 0]
output_op = ops

In [ ]:
ops

In [ ]:
len(iscell[iscell == True])

## Registration

In [ ]:
# Registration
plt.figure(figsize=(20, 5))
plt.subplot(1, 4, 1)
plt.imshow(
    output_op["refImg"],
    cmap="gray",
)
plt.title("Reference Image for Registration")

plt.subplot(1, 4, 2)
plt.imshow(output_op["max_proj"], cmap="gray")
plt.title("Registered Image, Max Projection")

plt.subplot(1, 4, 3)
plt.imshow(output_op["meanImg"], cmap="gray")
plt.title("Mean registered image")

plt.subplot(1, 4, 4)
plt.imshow(output_op["meanImgE"], cmap="gray")
plt.title("High-pass filtered Mean registered image");

## Detection

In [ ]:
# Detection
stats_file = Path(output_op["save_path"]).joinpath("stat.npy")
iscell = np.load(
    Path(output_op["save_path"]).joinpath("iscell.npy"), allow_pickle=True
)[:, 0].astype(bool)
stats = np.load(stats_file, allow_pickle=True)
stats.shape, iscell.shape

In [ ]:
# Make an array for ROI mask for each ROI
def stats_to_array(
    stats: Sequence[Dict[str, Any]], Ly: int, Lx: int, label_id: bool = False
):
    """
    converts stats sequence of dictionaries to an array
    :param stats: sequence of dictionaries from stat.npy
    :param Ly: number of pixels along dim Y from ops dictionary
    :param Lx: number of pixels along dim X
    :param label_id: keeps ROI indexing
    :return: numpy stack of arrays, each containing x and y pixels for each ROI
    """
    arrays = []
    for i, stat in enumerate(stats):
        arr = np.zeros((Ly, Lx), dtype=float)
        arr[stat["ypix"], stat["xpix"]] = 1
        if label_id:
            arr *= i + 1
        arrays.append(arr)
    return np.stack(arrays)


stats_arr = stats_to_array(stats, ops["Ly"], ops["Lx"])
stats_arr.shape

In [ ]:
def plot_detection_outcome(stats, ops, iscell, fname=None, output_dir=None):
    """
    generates a four panel plot with maximum intensity projection, both cell and non-cell ROIs
    detected in recording, all non-cell ROIs and all cell ROIs
    :param stats: stats array from stat.npy
    :param ops: dictionary of suite2p settings
    :param iscell: boolean array of which ROIs are identified as cells
    :param fname: name of recording for writing plots to file
    :param output_dir: path to directory for writing plots to file
    :return: none
    """
    im = stats_to_array(stats, Ly=ops["Ly"], Lx=ops["Lx"], label_id=True)
    im[im == 0] = np.nan

    plt.ioff()
    fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(20, 5))
    plt.subplot(1, 4, 1)
    plt.imshow(ops["max_proj"], cmap="gray")
    plt.title("registered image, max projection")

    plt.subplot(1, 4, 2)
    plt.imshow(np.nanmax(im, axis=0), cmap="jet")
    plt.title("all ROIs detected")

    plt.subplot(1, 4, 3)
    plt.imshow(np.nanmax(im[~iscell], axis=0), cmap="jet")
    plt.title("all non-cell ROIs")

    plt.subplot(1, 4, 4)
    plt.imshow(np.nanmax(im[iscell], axis=0), cmap="jet")
    plt.title("all cell ROIs")

    if (fname != None) and (output_dir != None):
        fig_name = Path(output_dir).joinpath("%s_cell-detect-outcomes.svg" % fname)

        fig.savefig(fig_name, format="svg", dpi=1200)
        plt.close(fig)


plot_detection_outcome(stats, ops, iscell)

## Extract Traces

In [ ]:
f_cells = np.load(Path(output_op["save_path"]).joinpath("F.npy"))
f_neus = np.load(Path(output_op["save_path"]).joinpath("Fneu.npy"))
spks = np.load(Path(output_op["save_path"]).joinpath("spks.npy"))
f_cells.shape, f_neus.shape, spks.shape

In [ ]:
f_cells = np.load(Path(trace_folder).joinpath("F.npy"))
f_neus = np.load(Path(trace_folder).joinpath("Fneu.npy"))
spks = np.load(Path(trace_folder).joinpath("spks.npy"))
f_ast = np.load(Path(trace_folder).joinpath("Fast.npy"))
f_cells.shape, f_neus.shape, spks.shape, f_ast.shape

In [ ]:
# Visualize one roi and its neuropil


def make_bounding_box(stat):
    """
    utility function for creating a bounding box around cells and neuropil masks
    :param stat: numpy array from stat.npy
    :returns y_lim1, y_lim2, x_lim1, x_lim2: x and y pixels for adding ~ 40 px border around cell ROI or mask
    """
    y_min = stat["ypix"].min()
    y_max = stat["ypix"].max()

    x_min = stat["xpix"].min()
    x_max = stat["xpix"].max()

    y_pad = round((80 - np.ptp(stat["ypix"])) / 2)
    x_pad = round((80 - np.ptp(stat["xpix"])) / 2)

    y_lim1 = y_min - y_pad
    y_lim2 = y_max + y_pad

    x_lim1 = x_min - x_pad
    x_lim2 = x_max + x_pad

    return y_lim1, y_lim2, x_lim1, x_lim2


def plot_roi_and_neuropil(
    f,
    f_neu,
    spks,
    ops,
    stat,
    which_roi,
    plot_start=0,
    plot_stop=f_cells.shape[1],
    fname=None,
    out_dir=None,
):
    """
    utility for generating a multipanel plot by user-selected ROI. plots
    fluoresence trace, neuropil fluo trace, deconvolved spikes, cell ROI and
    neuropil mask. f, f_neu and spks are only plotted for first 2000 frames
    :param f: numpy array of fluorescence values
    :param f_neu: numpy array of neuropil fluorescence values
    :param spks: numpy array of deconvolved spikes
    :param ops: dictionary of suite2p settings
    :param stat: numpy array of stat.npy
    :param which_roi: index of ROI to be plotted
    :param fname: string containing name of recording for writing figure to file
    :param out_dir: path to write .svg files
    :return: none
    """
    im = stats_to_array(stat, Ly=ops["Ly"], Lx=ops["Lx"], label_id=True)
    im[im == 0] = np.nan

    fig = plt.figure(figsize=(12, 4))
    grid = plt.GridSpec(3, 4, wspace=0.1, hspace=0.1, figure=fig)

    f_ax = fig.add_subplot(grid[0:2, 0:2])
    f_ax.plot(
        range(plot_start, plot_stop, 1),
        f[which_roi, range(plot_start, plot_stop, 1)],
        "g",
        alpha=0.8,
    )
    f_ax.plot(
        range(plot_start, plot_stop, 1),
        f_neu[which_roi, range(plot_start, plot_stop, 1)],
        "m",
        alpha=0.5,
    )
    f_ax.set_ylabel("fluorescence")

    # Adjust spks range to match range of fluorescence traces
    fmax = np.maximum(f.max(), f_neu.max())
    fmin = np.minimum(f.min(), f_neu.min())
    frange = fmax - fmin
    sp = spks[
        which_roi,
    ]
    sp /= sp.max()
    sp *= frange
    sp = sp[range(plot_start, plot_stop, 1)]

    spks_ax = fig.add_subplot(grid[2, 0:2])
    spks_ax.plot(range(plot_start, plot_stop, 1), sp, "k")
    spks_ax.set_xlabel("frame")
    spks_ax.set_ylabel("deconvolved")

    # Calculate bounding box for visualising ROI overlaid on meanImgE (consider writing as its own function)
    s = stat[which_roi]
    y_lim1, y_lim2, x_lim1, x_lim2 = make_bounding_box(s)

    img_ax = fig.add_subplot(grid[:, 2])
    img_ax.imshow(ops["meanImgE"][y_lim1:y_lim2, x_lim1:x_lim2], cmap="gray")
    img_ax.imshow(im[which_roi, y_lim1:y_lim2, x_lim1:x_lim2], alpha=0.5, cmap="spring")
    img_ax.title.set_text("ROI index %s" % (which_roi))

    cell_pix = np.zeros((ops["Ly"], ops["Lx"]))
    lammap = np.zeros((ops["Ly"], ops["Lx"]))
    ypix = s["ypix"]
    xpix = s["xpix"]
    lam = s["lam"]
    lammap[ypix, xpix] = np.maximum(lammap[ypix, xpix], lam)
    cell_pix = lammap > 0.0

    mask = suite2p.extraction.create_neuropil_masks(
        ypixs=s["ypix"],
        xpixs=s["xpix"],
        cell_pix=cell_pix,
        inner_neuropil_radius=ops["inner_neuropil_radius"],
        min_neuropil_pixels=ops["min_neuropil_pixels"],
        circular=ops.get("circular_neuropil", False),
    )

    neu_mask = np.zeros(512 * 512)
    neu_mask[mask[0]] = 1
    # check that pixels are being mapped back to 2D array correctly
    neu_mask = np.reshape(neu_mask, (512, 512))
    neu_mask[neu_mask == 0] = np.nan

    mask_ax = fig.add_subplot(grid[:, 3])
    mask_ax.imshow(ops["meanImgE"][y_lim1:y_lim2, x_lim1:x_lim2], cmap="gray")
    mask_ax.imshow(
        neu_mask[y_lim1:y_lim2, x_lim1:x_lim2],
        cmap=plt.cm.get_cmap("spring").reversed(),
        alpha=0.5,
    )
    mask_ax.title.set_text("neuropil mask")

    if (fname != None) and (output_dir != None):
        # fix potential errors if trailing slash is not included
        basename = fname + "_f-cell-neuropil_roi%s.svg" % which_roi
        fig_name = Path(out_dir).joinpath(basename)

        fig.savefig(fig_name, format="svg", dpi=1200)
        plt.close(fig)


# add function for plotting f vs f_neu for ROI of interest
def plot_f_f_neu(f, f_neu, which_roi, fname=None, out_dir=None):
    """
    a function that plots f vs f_neu for user-selected ROI, displays as a colour map of the KDE
    :param f: numpy array of cell fluorescence values
    :param f_neu: numpy array of neuropil fluorescence values
    :param which_roi: index of ROI
    :param fname: str, name of recording for writing figure to file
    :param out_dir: str, path to directory for writing .svg files
    :return: none
    """
    x = f[which_roi, :]
    y = f_neu[which_roi, :]
    # calculate Pearson's R from cell and neuropil fluorescence
    corr, _ = pearsonr(x, y)

    # create a gaussian KDE on a regular grid of 256 x 256 bins
    nbins = 256
    k = kde.gaussian_kde([x, y])
    xi, yi = np.mgrid[x.min() : x.max() : nbins * 1j, y.min() : y.max() : nbins * 1j]
    zi = k(np.vstack([xi.flatten(), yi.flatten()]))

    # make a figure of cell and neuropil fluorescence values, density
    # of points
    fig = plt.figure(figsize=(4, 4))
    ax = fig.gca()

    ax.tick_params(axis="y", left=True, which="major", labelleft=True)
    czset = ax.contourf(xi, yi, zi.reshape(xi.shape), cmap="Greens")
    cset = ax.contour(xi, yi, zi.reshape(xi.shape), colors="k")

    ax.clabel(cset, inline=1, fontsize=8)
    ax.set_xlabel("cell fluorescence")
    ax.set_ylabel("neuropil fluorescence")
    ax.set_title("ROI index %s" % which_roi)
    ax.text(
        0.1,
        0.9,
        "Pearson's R = %3f" % corr,
        verticalalignment="bottom",
        horizontalalignment="left",
        transform=ax.transAxes,
    )

    if (fname != None) and (output_dir != None):
        basename = fname + "_f-cell-neuropil-corr_roi%s.svg" % which_roi
        fig_name = Path(out_dir).joinpath(basename)

        fig.savefig(fig_name, format="svg", dpi=1200)
        plt.close(fig)


plot_roi_and_neuropil(
    F, Fneu, spks, ops, stats, which_roi=0, plot_start=0, plot_stop=2000
)

In [ ]:
# Plot selected roi traces in a certain time frame
def plot_roi_trace(
    f_cells,
    f_neus,
    spks,
    which_rois,
    plot_start=0,
    plot_end=f_cells.shape[1],
    enable_fcells=True,
    enable_fneus=True,
    enable_spks=True,
    lw=0.5,
    colors=["skyblue", "orange", "forestgreen"],
):

    plt.figure(figsize=[20, 5 * len(which_rois)])
    plt.suptitle("Flourescence and Deconvolved Traces for Different ROIs", y=0.92)
    for i, roi in enumerate(which_rois):
        plt.subplot(
            len(which_rois),
            1,
            i + 1,
        )
        f = f_cells[roi][plot_start:plot_end]
        f_neu = f_neus[roi][plot_start:plot_end]
        sp = spks[roi][plot_start:plot_end]
        # Adjust spks range to match range of fluroescence traces
        fmax = np.maximum(f.max(), f_neu.max())
        fmin = np.minimum(f.min(), f_neu.min())
        frange = fmax - fmin
        sp /= sp.max()
        sp *= frange
        if enable_fcells:
            plt.plot(f, label="Cell Fluorescence", color=colors[0], linewidth=lw)
        if enable_fneus:
            plt.plot(
                f_neu, label="Neuropil Fluorescence", color=colors[1], linewidth=lw
            )
        if enable_spks:
            plt.plot(sp + fmin, label="Deconvolved", color=colors[2], linewidth=lw)
        plt.xticks(np.arange(0, len(f), len(f) / 10))
        plt.ylabel(f"ROI {roi}", rotation=90)
        if i == 0:
            plt.legend(bbox_to_anchor=(0.93, 2))
    plt.xlabel("frame")


which_rois = np.arange(10)
plot_roi_trace(
    f_cells,
    f_neus,
    spks,
    which_rois,
    plot_start=0,
    plot_end=1000,
    enable_fcells=True,
    enable_fneus=True,
    enable_spks=True,
    colors=["skyblue", "orange", "forestgreen"],
)

# Process vis-stim loggers & harp

## Load & Format files & Synchronization

In [ ]:
# Filepath
def generate_rawdata_prefix(session, recording, protocol):
    return mouse + "_" + session + "_" + recording + "_" + protocol


rawdata_prefix = generate_rawdata_prefix(session, recording, protocol)
photodiode_file = rawdata_folder + rawdata_prefix + "_" + "PhotodiodeLog.csv"
rotary_encoder_file = rawdata_folder + rawdata_prefix + "_" + "RotaryEncoder.csv"
harpmessage_file = rawdata_folder + rawdata_prefix + "_" + "harpmessage.csv"
paramlog_file = rawdata_folder + rawdata_prefix + "_" + "NewParams.csv"

In [ ]:
# Find vis-stim frames
def format_VS_photodiode_logger(VS_photodiode_logger):
    """
    Format dataframe for VisStim frame_logger
    df columns = 'HarpTime','ElapsedTime'
    Parameters
    ----------
    VS_frame_logger : Dataframe
        Loaded dataframe from VS frame_logger.csv
    Returns
    -------
    formatted_df : Dataframe
        Formatted dataframe for VS_frame_logger
    """
    formatted_df = pd.DataFrame(columns=["HarpTime", "Photodiode"])
    formatted_df["HarpTime"] = VS_photodiode_logger["HarpTime"]
    formatted_df["Photodiode"] = VS_photodiode_logger["Photodiode"]

    formatted_df["ElapsedTime"] = (
        VS_photodiode_logger["HarpTime"] - VS_photodiode_logger.loc[0, "HarpTime"]
    )

    # Returns
    return formatted_df


VS_photodiode_logger = pd.read_csv(photodiode_file, sep=",")
VS_photodiode_logger = format_VS_photodiode_logger(VS_photodiode_logger)
plt.plot(VS_photodiode_logger["Photodiode"][10000:11000])
VS_photodiode_logger = find_frames.find_VS_frames(
    photodiode_df=VS_photodiode_logger,
    frame_rate=144,
    upper_thr=180,
    lower_thr=25,
    plot=True,
    plot_start=10000,
    plot_range=2000,
    plot_dir=None,
)

In [ ]:
# Load params
def format_VS_param_logger(VS_param_logger, which_protocol):
    """
    Format dataframe for VisStim param_logger
    df columns = 'HarpTime','ElapsedTime' (calculated from the start of frame logger!!), Params...

    Parameters
    ----------
    VS_param_logger : Dataframe
        Loaded dataframe from VS param_logger.csv
    VS_frame_logger : Dataframe
        Loaded dataframe from VS frame_logger.csv
    which_protocol: string
        The Vis-Stim protocol used ('Retinotopic','Fourier')
    Returns
    -------
    formatted_df : Dataframe
        Formatted dataframe for VS_param_logger. !!ElapsedTime is already alighed to the starting of the frame logger
    """
    formatted_df = pd.DataFrame(columns=["HarpTime"])

    formatted_df["HarpTime"] = VS_param_logger["HarpTime"]
    # formatted_df['ElapsedTime'] = VS_param_logger['HarpTime'] - VS_frame_logger.loc[0,'HarpTime']

    # Assign params according to stimulation protocol
    if which_protocol == "Retinotopy":
        formatted_df["Xdeg"] = VS_param_logger["Xdeg"]
        formatted_df["Ydeg"] = VS_param_logger["Ydeg"]
        formatted_df["Angle"] = VS_param_logger["Angle"]

    elif which_protocol == "Fourier":
        formatted_df["BarID"] = VS_param_logger["BarID"]
        formatted_df["LocationX"] = VS_param_logger["LocationX"]
        formatted_df["LocationY"] = VS_param_logger["LocationY"]
        formatted_df["Angle"] = VS_param_logger["Angle"]

    elif which_protocol == "Episodic":
        formatted_df["StimID"] = VS_param_logger["StimID"]
        formatted_df["Azimuth"] = VS_param_logger["Azimuth"]
        formatted_df["Elevation"] = VS_param_logger["Elevation"]
        formatted_df["Angle"] = VS_param_logger["Angle"]

    elif which_protocol == "SphereSparseNoise":
        formatted_df["SphereID"] = VS_param_logger["SphereID"]
        formatted_df["Depth"] = VS_param_logger["Depth"]
        formatted_df["Azimuth"] = VS_param_logger["Azimuth"]
        formatted_df["Elevation"] = VS_param_logger["Elevation"]

    # Returns
    return formatted_df


VS_param_logger = pd.read_csv(paramlog_file, sep=",")
VS_param_logger = format_VS_param_logger(
    VS_param_logger=VS_param_logger, which_protocol=protocol
)

In [ ]:
# Find frame triggers for imaging
harp_message = pd.read_csv(
    harpmessage_file, sep=",", usecols=["RegisterAddress", "Timestamp"]
)
img_frame_logger = find_frames.find_imaging_frames(
    harp_message=harp_message,
    frame_number=f_cells.shape[1],
    exposure_time=0.0324,
    register_address=32,
    exposure_time_tolerance=0.001,
)
img_frame_logger

In [ ]:
# Align timestamps

img_VS = align_timestamps.align_timestamps(
    VS_frames=VS_photodiode_logger,
    VS_params=VS_param_logger,
    imaging_frames=img_frame_logger,
)
img_VS

In [ ]:
# Add running logger
mousez_logger = pd.read_csv(rotary_encoder_file, sep=",", usecols=["HarpTime", "EyeZ"])
img_VS = pd.merge_asof(
    img_VS, mousez_logger, on="HarpTime", allow_exact_matches=True, direction="backward"
)

img_VS.EyeZ = img_VS.EyeZ / 100  # Convert cm to m
img_VS.Depth = img_VS.Depth / 100  # Convert cm to m
img_VS

In [ ]:
# save img_VS
with open(protocol_folder + "img_VS.pickle", "wb") as handle:
    pickle.dump(img_VS, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Process vis-stim info

In [ ]:
with open(protocol_folder + "img_VS.pickle", "rb") as handle:
    img_VS = pickle.load(handle)

img_VS["Stim"] = np.nan
img_VS.loc[img_VS.Depth.notnull(), "Stim"] = 1
img_VS.loc[img_VS.Depth < 0, "Stim"] = 0
img_VS.loc[((img_VS[img_VS.Depth < 0]).index.values - 1), "Stim"] = 0

img_VS_simple = img_VS[(img_VS["Stim"].diff() != 0) & (img_VS["Stim"].notnull())]
img_VS_simple.Depth = np.round(img_VS_simple.Depth, 2)
img_VS_simple

In [ ]:
# Find the frame number of each vis-stim trial
depth_list = [0.2, 0.63, 2]
stim_dict = {}
for istim in depth_list:
    stim_dict["stim" + str(istim)] = {}
    stim_dict["stim" + str(istim)]["start"] = img_VS_simple[
        (img_VS_simple["Depth"] == istim) & (img_VS_simple["Stim"] == 1)
    ].index.values
    stim_dict["stim" + str(istim)]["stop"] = img_VS_simple[
        (img_VS_simple["Depth"] == istim) & (img_VS_simple["Stim"] == 0)
    ].index.values
stim_dict

In [ ]:
# Find frames for blank period
blank_points = img_VS_simple.index.values[1:-1]
stim_dict["blank"] = {}
stim_dict["blank"]["start"] = blank_points[0::2] + 1
stim_dict["blank"]["stop"] = blank_points[1::2] - 1
stim_dict